# Analyzing Freddie Mac Single Family Loans

## Abstract
The Federal Home Loan Mortgage Corporation (FHLMC), known as Freddie Mac, is a public government-sponsored enterprise which was created to expand the secondary market for mortgages in the United States. Freddie Mac buys mortgages on the secondary market, pools them, and sells them as a mortgage-backed security to investors on the open market. This secondary mortgage market increases the supply of money available for mortgage lending and increases the money available for new home purchases.  
<br>
At the direction of its regulator, the Federal Housing Finance Agency (FHFA), Freddie Mac has made the Single Family Loan-Level Dataset (the "Dataset") available as part of a larger effort to increase transparency and help investors build mre accurate credit performance models in support of ongoing and future credit risk-sharing transactions. The Dataset includes: <br>
1. Loan-level origination, monthly loan performance, and actual loss data on a portion of the fully amortizing 30-year fixed-rate Single Family mortgages that Freddie Mac acquired with origination dates from 11999 to the Origination Cutoff Date. 
2. Loan-level origination, monthly loan performance, and actual loss data on a portion of the fully amortizing 15- and 20-year fixed-rate Single Family mortgages that Freddie Mac acquired with origination dates from January 1, 2005, to the Origination Cutoff Date. 
Loan performance information in the Dataset includes the monthly loan balance, delinquency status and certain information up to and including the earliest of the following termination events: <br>
    a. Prepaid or Matured (voluntary Payoff) <br>
    b. Foreclosure Alternative Group (Short Sale, Third Party Sale, Charge Off or Note Sale) <br>
    c. Repurchase prior to Property Disposition. <br>
    d. REO Disposition <br>

## Single Family Loan-Level Dataset Sample
Freddie Mac has created a smaller dataset which is a simple random sample of 50,000 loans selected from each full vintage year and a proportionate nubmer of loans from each partial vintage year of the full Single Family Loan-Level Dataset. Each vintage year has one origination data file and one corresponding monthly performance data file, containing the same loan-level data fields as those included in the full Dataset.

## Hypothesis
The outcome of housing loan defaults can be statistically modeled and predicted as a function of financial factors, specifically interest Rate and consumer credit history.  

## Dataset Characteristics

#### Time period: 2012 to 2016
#### Number of loans:
#### Features:

# Initial Data Import & Treatment
Origination and performance data must be downloaded individually by year from the Freddie Mac website. <br>
<br>
Format as follows: <br>
sample_orig_YYYY.txt --> origination data <br>
sample_svcg_YYYY.txt --> monthly performance data <br>
<br>
Both origination and performance files share the common "loan sequence number" which serves as the unique loan identifier. The "loan sequence number" includes the year and quarter (Q1, Q2, etc) of loan origination. The performance file shows monthly performance for each loan, thus there are multiple rows (month1, month2, etc) corresponding to each "loan sequence number". There are multiple null values and data-types within the datasets, and the data is considered "living", meaning it is subject to change as Freddie Mac receives & updates to their housing loan profiles.  

In [42]:
# Import Modules:
import time
import math
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Aesthetics.
%matplotlib inline
sns.set_style('white')
#fig, ax = plt.subplots(1, 2, figsize=(18, 4))

In [43]:
# Load raw files.
orig2012 = pd.read_csv('~/src/data/u3CapstoneData/origination-data/sample_orig_2012.txt', sep='|', header=None)
orig2013 = pd.read_csv('~/src/data/u3CapstoneData/origination-data/sample_orig_2013.txt', sep='|', header=None)
orig2014 = pd.read_csv('~/src/data/u3CapstoneData/origination-data/sample_orig_2014.txt', sep='|', header=None)
orig2015 = pd.read_csv('~/src/data/u3CapstoneData/origination-data/sample_orig_2015.txt', sep='|', header=None)
orig2016 = pd.read_csv('~/src/data/u3CapstoneData/origination-data/sample_orig_2016.txt', sep='|', header=None)

perf2012 = pd.read_csv('~/src/data/u3CapstoneData/perf-data/sample_svcg_2012.txt', sep='|', header=None, low_memory=False)
perf2013 = pd.read_csv('~/src/data/u3CapstoneData/perf-data/sample_svcg_2013.txt', sep='|', header=None, low_memory=False)
perf2014 = pd.read_csv('~/src/data/u3CapstoneData/perf-data/sample_svcg_2014.txt', sep='|', header=None, low_memory=False)
perf2015 = pd.read_csv('~/src/data/u3CapstoneData/perf-data/sample_svcg_2015.txt', sep='|', header=None, low_memory=False)
perf2016 = pd.read_csv('~/src/data/u3CapstoneData/perf-data/sample_svcg_2016.txt', sep='|', header=None, low_memory=False)

In [44]:
# Setting header names & apply.
orig_cols = ['creditScore', 'firstPaymentDate', 'firstTimeHomebuyerFlag', 'maturityDate',
                   'msa', 'mortagageInsurancePercentage', 'numberOfUnits',
                   'occupancyStatus', 'cltvRatio', 'dtiRatio', 'upb',
                   'ltvRatio', 'interestRate', 'channel', 'ppmFlag', 'productType',
                   'propertyState', 'propertyType', 'postalCode', 'lsn',
                   'loanPurpose', 'originalLoanTerm', 'numberOfBorrowers', 'sellerName',
                   'servicerName', 'superConformingFlag'] # 'pre-HarpLoanSequenceNumber'

perf_cols = ['lsn', 'monthlyReportingPeriod', 'currentActualUpb',
                 'currentLoanDelinquencyStatus', 'loanAge', 'remainMthsToMaturity', 'repurchaseFlag',
                 'modificationFlag', 'zeroBalCode', 'zeroBalEffDate', 'currentIntRate',
                 'curDeferredUpb', 'ddlpi', 'miRecov', 'netSalesProceeds', 'nonMiRecov',
                 'expenses', 'legalCosts', 'maintPreservationCosts', 'taxesInsurance',
                 'miscExpenses', 'actualLossCalc', 'modificationCost',
                 'stepModificationFlag', 'deferredPaymentModification']

orig2012.columns = orig_cols
orig2013.columns = orig_cols
orig2014.columns = orig_cols
orig2015.columns = orig_cols
orig2016.columns = orig_cols

perf2012.columns = perf_cols
perf2013.columns = perf_cols
perf2014.columns = perf_cols
perf2015.columns = perf_cols
perf2016.columns = perf_cols

In [45]:
# Merge origination files.
frames_orig = [orig2012, orig2013, orig2014, orig2015, orig2016]
orig_combined = pd.concat(frames_orig)
print(orig_combined.shape)
#orig_combined.head()

(250000, 26)


In [46]:
# Merge performance files.
frames_perf = [perf2012, perf2013, perf2014, perf2015, perf2016]
perf_combined = pd.concat(frames_perf)
print(perf_combined.shape)
#perf_combined.head()

(8727988, 25)


In [47]:
# Create a 'year' column within the merged performance file.

# The loan sequence number 'lsn' is formatted F1YYQnXXXXXX,
# where F1 refers to the product 'Fixed Rate Mortgage',
# YYQn refers to origination year and quarter.

perf_combined['year'] = ['19' + x if x == '99' else '20' + x for x in (perf_combined['lsn'].apply(lambda x: x[2:4]))]
perf_combined.head()

,lsn,monthlyReportingPeriod,currentActualUpb,currentLoanDelinquencyStatus,loanAge,remainMthsToMaturity,repurchaseFlag,modificationFlag,zeroBalCode,zeroBalEffDate,...,expenses,legalCosts,maintPreservationCosts,taxesInsurance,miscExpenses,actualLossCalc,modificationCost,stepModificationFlag,deferredPaymentModification,year
0,F112Q1000057,201202,103000.0,0,0,360,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,2012
1,F112Q1000057,201203,103000.0,0,1,359,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,2012
2,F112Q1000057,201204,103000.0,0,2,358,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,2012
3,F112Q1000057,201205,102000.0,0,3,357,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,2012
4,F112Q1000057,201206,102000.0,0,4,356,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,2012


In [48]:
# Merge combined origination & combined performance dataframes.
df_merged = pd.merge(orig_combined, perf_combined, on='lsn', how='inner')
print(df_merged.columns)
df_merged.head()

Index(['creditScore', 'firstPaymentDate', 'firstTimeHomebuyerFlag',
       'maturityDate', 'msa', 'mortagageInsurancePercentage', 'numberOfUnits',
       'occupancyStatus', 'cltvRatio', 'dtiRatio', 'upb', 'ltvRatio',
       'interestRate', 'channel', 'ppmFlag', 'productType', 'propertyState',
       'propertyType', 'postalCode', 'lsn', 'loanPurpose', 'originalLoanTerm',
       'numberOfBorrowers', 'sellerName', 'servicerName',
       'superConformingFlag', 'monthlyReportingPeriod', 'currentActualUpb',
       'currentLoanDelinquencyStatus', 'loanAge', 'remainMthsToMaturity',
       'repurchaseFlag', 'modificationFlag', 'zeroBalCode', 'zeroBalEffDate',
       'currentIntRate', 'curDeferredUpb', 'ddlpi', 'miRecov',
       'netSalesProceeds', 'nonMiRecov', 'expenses', 'legalCosts',
       'maintPreservationCosts', 'taxesInsurance', 'miscExpenses',
       'actualLossCalc', 'modificationCost', 'stepModificationFlag',
       'deferredPaymentModification', 'year'],
      dtype='object')


,creditScore,firstPaymentDate,firstTimeHomebuyerFlag,maturityDate,msa,mortagageInsurancePercentage,numberOfUnits,occupancyStatus,cltvRatio,dtiRatio,...,expenses,legalCosts,maintPreservationCosts,taxesInsurance,miscExpenses,actualLossCalc,modificationCost,stepModificationFlag,deferredPaymentModification,year
0,814,201203,9,204202,49420.0,0,1,P,57,36,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,2012
1,814,201203,9,204202,49420.0,0,1,P,57,36,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,2012
2,814,201203,9,204202,49420.0,0,1,P,57,36,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,2012
3,814,201203,9,204202,49420.0,0,1,P,57,36,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,2012
4,814,201203,9,204202,49420.0,0,1,P,57,36,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,2012


In [49]:
# Reordering columns.
lsn_var = df_merged['lsn']
year_var = df_merged['year']
monthlyReportingPeriod_var = df_merged['monthlyReportingPeriod']
currentLoanDelinquencyStatus_var = df_merged['currentLoanDelinquencyStatus']

df_merged.drop(['lsn', 'year', 'monthlyReportingPeriod', 'currentLoanDelinquencyStatus'], axis=1, inplace=True)

df_merged.insert(0, 'lsn', lsn_var)
df_merged.insert(1, 'year', year_var)
df_merged.insert(2, 'monthlyReportingPeriod', monthlyReportingPeriod_var)
df_merged.insert(3, 'currentLoanDelinquencyStatus', currentLoanDelinquencyStatus_var)
print(df_merged.columns)
print(df_merged.shape)
df_merged.head()

Index(['lsn', 'year', 'monthlyReportingPeriod', 'currentLoanDelinquencyStatus',
       'creditScore', 'firstPaymentDate', 'firstTimeHomebuyerFlag',
       'maturityDate', 'msa', 'mortagageInsurancePercentage', 'numberOfUnits',
       'occupancyStatus', 'cltvRatio', 'dtiRatio', 'upb', 'ltvRatio',
       'interestRate', 'channel', 'ppmFlag', 'productType', 'propertyState',
       'propertyType', 'postalCode', 'loanPurpose', 'originalLoanTerm',
       'numberOfBorrowers', 'sellerName', 'servicerName',
       'superConformingFlag', 'currentActualUpb', 'loanAge',
       'remainMthsToMaturity', 'repurchaseFlag', 'modificationFlag',
       'zeroBalCode', 'zeroBalEffDate', 'currentIntRate', 'curDeferredUpb',
       'ddlpi', 'miRecov', 'netSalesProceeds', 'nonMiRecov', 'expenses',
       'legalCosts', 'maintPreservationCosts', 'taxesInsurance',
       'miscExpenses', 'actualLossCalc', 'modificationCost',
       'stepModificationFlag', 'deferredPaymentModification'],
      dtype='object')
(87279

,lsn,year,monthlyReportingPeriod,currentLoanDelinquencyStatus,creditScore,firstPaymentDate,firstTimeHomebuyerFlag,maturityDate,msa,mortagageInsurancePercentage,...,nonMiRecov,expenses,legalCosts,maintPreservationCosts,taxesInsurance,miscExpenses,actualLossCalc,modificationCost,stepModificationFlag,deferredPaymentModification
0,F112Q1000057,2012,201202,0,814,201203,9,204202,49420.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
1,F112Q1000057,2012,201203,0,814,201203,9,204202,49420.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
2,F112Q1000057,2012,201204,0,814,201203,9,204202,49420.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
3,F112Q1000057,2012,201205,0,814,201203,9,204202,49420.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
4,F112Q1000057,2012,201206,0,814,201203,9,204202,49420.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,


# COMBINING ORIG & PERF COMPLETE

In [75]:
# Creating copy.
df_raw = df_merged.copy()

In [80]:
df_raw.head(200)

,lsn,year,monthlyReportingPeriod,currentLoanDelinquencyStatus,creditScore,firstPaymentDate,firstTimeHomebuyerFlag,maturityDate,msa,mortagageInsurancePercentage,...,nonMiRecov,expenses,legalCosts,maintPreservationCosts,taxesInsurance,miscExpenses,actualLossCalc,modificationCost,stepModificationFlag,deferredPaymentModification
0,F112Q1000057,2012,201202,0,814,201203,9,204202,49420.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
1,F112Q1000057,2012,201203,0,814,201203,9,204202,49420.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
2,F112Q1000057,2012,201204,0,814,201203,9,204202,49420.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
3,F112Q1000057,2012,201205,0,814,201203,9,204202,49420.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
4,F112Q1000057,2012,201206,0,814,201203,9,204202,49420.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
5,F112Q1000057,2012,201207,0,814,201203,9,204202,49420.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
6,F112Q1000057,2012,201208,0,814,201203,9,204202,49420.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
7,F112Q1000057,2012,201209,0,814,201203,9,204202,49420.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
8,F112Q1000057,2012,201210,0,814,201203,9,204202,49420.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
9,F112Q1000057,2012,201211,0,814,201203,9,204202,49420.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,


In [79]:
# View dtypes
df_raw.dtypes

lsn                              object
year                             object
monthlyReportingPeriod            int64
currentLoanDelinquencyStatus     object
creditScore                       int64
firstPaymentDate                  int64
firstTimeHomebuyerFlag           object
maturityDate                      int64
msa                             float64
mortagageInsurancePercentage      int64
numberOfUnits                     int64
occupancyStatus                  object
cltvRatio                         int64
dtiRatio                          int64
upb                               int64
ltvRatio                          int64
interestRate                    float64
channel                          object
ppmFlag                          object
productType                      object
propertyState                    object
propertyType                     object
postalCode                        int64
loanPurpose                      object
originalLoanTerm                  int64


In [ ]:
def changeDataType(df_raw):

In [81]:
df_raw.currentLoanDelinquencyStatus.value_counts()

0     7857555
0      830116
1       25190
2        4436
1        2774
3        1793
4        1093
5         873
6         602
7         465
2         427
8         372
9         325
10        263
11        211
12        190
3         153
13        152
14        117
R         101
15         85
16         78
17         67
4          65
18         51
5          43
19         39
20         32
21         29
25         26
       ...   
8          15
27         13
10         12
9          11
30         10
28         10
29          9
31          6
32          6
33          5
36          4
35          4
34          4
11          4
12          3
37          3
XX          2
13          2
40          2
41          2
38          2
39          2
14          1
43          1
44          1
46          1
45          1
47          1
42          1
48          1
Name: currentLoanDelinquencyStatus, Length: 66, dtype: int64